Разберем простые рекомендательные системы. Для этого возьмем датасет с рейтингами аниме и попробуем посоветовать пользователям сайта, что же им еще можно посмотеть.

Импортируем необходимые библиотеки.

## Обработка данных

In [ ]:
import pandas as pd
import numpy as np
import scipy.sparse as scp

Загрузим датасет с аниме, их названием, рейтингами

In [ ]:
anime = pd.read_csv('./anime.csv')

Рассмотрим только те аниме, которые входят в категорию TV

In [ ]:
anime = anime.loc[anime.type == 'TV']
anime.head()

Так же загрузим датасет с рейтигами, который пользователи ставили определенным аниме

In [ ]:
rating = pd.read_csv('./rating.csv')
rating.head()

Как видим, оба датасета имеют поле `anime_id`. Можем по этому полю соединить обе таблицы. Получим таблицу `merged`, в которой хранится, какой пользователь поставил какую оценку конкретному аниме.

Заметьте, что в обоих таблицах есть поле `rating`, поэтому при соединении таблиц изменим название этого поля в таблице `rating` на `rating_user`, добавив суффикс `_user`

In [ ]:
merged = rating.merge(anime, left_on='anime_id', right_on='anime_id', suffixes=['_user', ''])
merged.head()

Нам не нужно знать все о получившийся таблице. Главное знать id пользователя, название аниме и саму оценку. Оставим только нужные нам поля.

In [ ]:
merged = merged[['user_id', 'name', 'rating_user']]
merged.head()

Изменим навзание поля `rating_user` на более удобное для понимания: `user_rating`

In [ ]:
merged.rename(columns={'rating_user': 'user_rating'}, inplace=True)
merged.head()

Рейтинг `-1` значит, что пользователь посмотел аниме, но не поставил ему оценку. Так как нас интересуют только рейтинги, можно выкинуть все `-1`

In [ ]:
merged = merged.loc[merged.user_rating != -1]
merged.head()

Выкинем часть датасета, чтобы избежать проблем с вычислениями.

In [ ]:
merged = merged.loc[merged.user_id <= 10000]
merged.head()

Теперь разобьем данные на train и test части. Это не самый лучший способ разбить датасет на части, но сейчас для простоты используем его.

In [ ]:
from sklearn.model_selection import train_test_split

random_state = 314159

In [ ]:
train_data, dev_data = train_test_split(merged, test_size=0.1, random_state=random_state)
train_data.sort_index(inplace=True)
dev_data.sort_index(inplace=True)
train_data.shape, dev_data.shape

Train часть датасета

In [ ]:
train_data.head()

Test часть датасета

In [ ]:
dev_data.head()

In [ ]:
if len(merged.user_id.unique()) == merged.user_id.max():
    print('Количество уникальных пользователей совпадает с максимальным id пользователя')
else:
    print('Количество уникальных пользователей НЕ совпадает с максимальным id пользователя')

In [ ]:
merged.user_id.unique()

Как можно видеть, количество уникальных пользователей не совпадает с максимальным id пользователя, а это значит, что id пользователей сдвинуты. Так, например, отсутствуют пользователи, с `id` 0, 1, 2  и 4.

В дальнейшем нам надо будет работать с матрицей рейтингов, где 0-ая строка соответствует пользователю с `id` 3, 1-ая строка -- пользователю с `id` 5 и так далее. Поэтому для удобства создадим словарь, который переводит настоящий id пользователя в удобный нам. Сразу же сделаем словарь, который делает обратное преобразование.

In [ ]:
user_to_idx = {user_id : idx for idx, user_id in enumerate(merged.user_id.unique())}
idx_to_user = {i: user for user, i in user_to_idx.items()}

In [ ]:
assert 9387 == len(user_to_idx)

Как видно, первый словать перевел пользователя с id 5 в пользователя с id 1. То, что нам и нужно было

In [ ]:
user_to_idx[5]

Обратное преобразование так же работает

In [ ]:
idx_to_user[1]

Так же необходимо сопоставить именам аниме индексацию 

In [ ]:
anime_to_idx = {anime_name : idx for idx, anime_name in enumerate(merged.name.unique())}
idx_to_anime = {i: anime_name for anime_name, i in anime_to_idx.items()}

In [ ]:
assert 2708 == len(anime_to_idx)

Так можно видеть, что 0-ому столбцу соответсвует Naruto

In [ ]:
idx_to_anime[0]

Преобразуем train часть датасета с помощью преобразований, описанных выше.

In [ ]:
data = train_data.copy()
data['user_id'] = data['user_id'].apply(lambda x: user_to_idx[x])
data['name'] = data['name'].apply(lambda x: anime_to_idx[x])
data.head()

## Создание матрицы рейтингов

Теперь мы готовы для того, чтобы составить саму матрицу рейтингов. Если мы будем хранить все значения, то есть не только сами оценки пользователей, но так же и нули, говорящие о том, что оценки не было, то в итоге такая матрица займет слишком много места. Вместо этого используют размереженные матрицы.

Конкретно сейчас, будем работать с матрицей типа `coo_matrix`. Для того, чтобы созадть такую матрицу, достаточно знать координаты каждого ненулевого элемента. В нашей рекомендательной системе координатами являются id пользователей и id аниме.

<img src='http://cdncontribute.geeksforgeeks.org/wp-content/uploads/Sparse-Matrix-Array-Representation1.png'>

Так же нужно указать размеры создаваемой матрицы. У нас есть пользователи, которые не поставили ни одной оценки, а так же есть аниме, которым не ставили ни одной оценки. Чтобы случайно не выкинуть таких пользователей и такие аниме, зафиксируем размеры матрицы рейтингов.

In [ ]:
def get_sparse(data):
    return scp.coo_matrix(
        (
            data['user_rating'],  # оценки пользователей
            (data['user_id'], data['name'])  # id пользователей и id аниме, для которых известны оценки
        ), 
        shape=(len(user_to_idx), len(anime_to_idx))  # размеры матрицы рейтингов
    ).tocsr()

In [ ]:
train_sp = get_sparse(data)
train_sp

Можем перевести получившуюся `sparce` матрицу обратно к `dense` формату. Видно, что все проставленные оценки расположились так, что если оценка `r` стоит в `i` строке и `j` столбце, то значит пользователь с id `i` поставил аниме с id `j` оценку `r`. Все остальные поля заполнены нулями.

In [ ]:
train_dense = train_sp.todense().A
train_dense

Еще раз повторюсь: правильно работать с матрицей рейтингов, которая при этом является разреженной матрицей. Но так как мы пока что учимся, далее будем использовать знакомый нам тип: `dense`.

## Baseline prediction

Для начала попробуем предсказывать рейтинги пользователей простым способом. Найдем средний рейтинг для всех пользователей и всех аниме. Для этого просуммируем все проставленные рейтинги и поделим на их количество.
$$
\begin{align*}
\mu=&\frac{1}{n}\sum_{u,i}r_{ui}
\end{align*}
$$
Уже можно было бы сказать, что неизвестная нам оценка, которую поставит некий пользователь аниме, равна среднему. Но мы пойдем чуть дальше. Будем учитывать предвзятость пользователей и переоценку/недооценку аниме. Как пример, некоторые пользователи ставят в основном 10, а 9 для них - ужасный фильм, для некоторых пользователей наблюдается обратная ситуация: даже 5 для них - это шедевр кинематографа. С аниме такая же ситуация.

Сначала разберемся с пользователями. Для того, чтобы вычислить "сдвиг" в оценках пользователя $u$ относительно средней оценки $\mu$, достаточно вычесть из всех его оценок $r_{ui}$ среднюю оценку $\mu$, просуммировать результат и поделить на количество оценкок $|I_u|$, который этот пользователь поставил. Так как не все пользователи ставили оценки, то введем параметр $\alpha$, чтобы избежать деления на 0. $\alpha$ можно взять равным 1, но так как $\alpha$ еще является и коэффициентом сглаживания, то возьмем его равным $25$.
$$
\begin{align*}
b_u=&\frac{1}{|I_u|+\alpha}\sum_{i\in{}I_u}(r_{ui} - \mu), \space\space\space \alpha=25
\end{align*}
$$
Аналогично вычислим "сдвиг" в оценивании аниме $i$, но на этот раз учтем еще и найденные ранее сдвиги в оценках пользователей. Так же введем коэффициент сглаживания $\beta$ и примем его равным коэффициенту $\alpha$.
$$
\begin{align*}
b_i=&\frac{1}{|U_i|+\beta}\sum_{u^\prime\in{}U_i}(r_{u^\prime{}i} - b_{u^\prime} - \mu), \space\space\space \beta=25
\end{align*}
$$
Итоговое предсказание оценки $r_{ui}$ пользователя $u$ аниме $i$
$$
\begin{align*}
\hat{r}_{ui}=&\mu+b_u+b_i
\end{align*}
$$

In [ ]:
alpha = beta = 25

Так как при вычислении среднего рейтинга нам надо учитывать только известные оценки, то используем формат `masked_array`, который "скроет" все нули. Такой формат представляет из себя 2 поля: `data` и `mask`. В `data` хранятся все необходимые данные (в нашем случае матрица рейтингов), а в `mask` указывается, какие значения из `data` учитываются: True - значение не учитывается, False - учитывается.

In [ ]:
import numpy.ma as ma

In [ ]:
train_dense_masked = ma.masked_array(train_dense, mask=train_dense == 0, fill_value=0)
train_dense_masked

Вычислим среднюю оценку. Довольно высокая :)

In [ ]:
mu = train_dense_masked.mean()
mu

С учетом средней оценки вычислим "сдвиг" в оценке для каждого пользователя по формуле, описанной выше.

Уже видно, что, например, второй пользователь в среднем ставит оценку на 3 балла меньше, чем средняя оценка всех пользователей. То есть там, где обычный пользователь поставил бы 8, он поставит 5. Именно из-за таких пользователей и следует делать поправку на предвзятость.

In [ ]:
bu = (train_dense_masked - mu).sum(1)/((~train_dense_masked.mask).sum(1) + alpha)
bu.fill_value = 0
assert bu.shape == (len(user_to_idx), )
bu

Так же вычислим "сдвиг" для аниме. Как вы помните, выше выяснили, что для нашей индексации аниме с индексом 0 - это "Наруто". Как видим, нулевой элемент вектора сдвига для аниме равен $\approx{-0.09}$, то есть это насколько средняя оценка "Наруто" на 0.09 меньше средней оценки $\mu$.

In [ ]:
bi = (train_dense_masked - bu[..., None] - mu).sum(0)/((~train_dense_masked.mask).sum(0) + beta)
bi.fill_value = 0
assert bi.shape == (len(anime_to_idx), )
bi

В итоге объединим "сдвиги" пользователей и "сдвиги" аниме в единую матрицу. Тогда значение $b_{ui}$, находящееся на пересечении строки $u$ и столбца $i$, будет равно "сдвигу" оценивания аниме $i$ пользователем $u$ относительно средней оценки $\mu$.

In [ ]:
B = bu[..., None] + bi  # or np.outer(bu, bi)

Сложим среднюю оценку $\mu$ и все известные нам сдвиги $B$ в соответствии с формулой, описанной выше, и получим нашу простую рекомендательную систему. Значение $r_{ui}$, находящееся на пересечении строки $u$ и столбца $i$ матрицы baseline_predictions, оценка, которую поставит пользователь $u$ аниме $i$.

In [ ]:
baseline_predictions = mu + B
baseline_predictions

Посмотрим, насколько хорошо наша модель справляется. Для этого сгруппируем все оценки пользователей в train и test датасетах

In [ ]:
train_grby_user = train_data.groupby('user_id')
dev_grby_user = dev_data.groupby('user_id')

Выведем, какие оценки нам известны для пользователя с id 18

In [ ]:
# 16, 18, 19, 20
user = 18
train_grby_user.get_group(user)

Так же посмотрим, что он еще посмотрел и поставил оценки, но нам о них ничего не было известно на момент построения baseline prediction, так как они в тестовой выборке

In [ ]:
dev_grby_user.get_group(user)

Напишем небольшую функцию, которая вернет `top_k` аниме, которые должны понравиться пользователю с id `user`, а так же их прогнозируемые оценки

In [ ]:
def line_on_baseline(user, top_k=10):
    # отсортируем все рейтинги по убыванию
    ratings = baseline_predictions[user_to_idx[user]]
        
    anime_to_line_on = []
    for anime_idx in np.argsort(ratings)[::-1]:
        # пропустим аниме, которые пользователь уже смотрел
        if ma.is_masked(ratings[anime_idx]):
            continue

        # если нашли аниме, которое пользователь не смотрел, то добавим его в список
        if idx_to_anime[anime_idx] not in train_grby_user.get_group(user).name.values:
            anime_to_line_on.append((idx_to_anime[anime_idx], ratings[anime_idx]))

        # если список аниме, которые понравятся, уже содержит top_k фильмов, то прекратим поиск
        if len(anime_to_line_on) == top_k:
            break
            
    return anime_to_line_on

In [ ]:
dev_grby_user.get_group(16)

In [ ]:
line_on_baseline(16)

In [ ]:
dev_grby_user.get_group(18)

In [ ]:
line_on_baseline(18)

## Использование схожести пользователей

До этого мы считали рейтинг пользователей отталкиваясь от среднего рейтинга всех пользователей. Поэтому даже если пользователю $u$ нравится только жанр ужасы, мы при вычислении его оценок так же учитывали оценки тех пользователей, кто больше любит жанр комедии, что неправильно. Поэтому модефицируем наш простой алгоритм: во-первых, будем учитывать только $N$ похожих пользователей (множество $U_{sim}$); во-вторых, каждый рейтинг будем учитывать с весом равным схожести пользователей.
$$
\hat{r}_{ui}=\frac{\sum_{u^\prime \in U_{sim}}sim(u,u^\prime)r_{u^\prime{}i}}{\sum_{u^\prime \in U_{sim}}|sim(u,u^\prime)|}
$$
Немного улучшим алгоритм, используя уже знакомый вам подход: усредним не оценки похожих пользователей, а их "сдвиги" в оценках, и добавим это к средней оценке пользователя $u$.
$$
\hat{r}_{ui}=\bar{r}_u+\frac{\sum_{u^\prime}sim(u,u^\prime)(r_{u^\prime{}i}-\bar{r}_{u^\prime})}{\sum_{u^\prime}|sim(u,u^\prime)|}
$$
Здесь $\bar{r}_u$ - средний рейтинг пользователя $u$.

Используем косинусную близость:
$$
{\displaystyle {\text{similarity}}=\cos(\theta )={\mathbf {A} \cdot \mathbf {B}  \over \|\mathbf {A} \|\|\mathbf {B} \|}={\frac {\sum \limits _{i=1}^{n}{A_{i}B_{i}}}{{\sqrt {\sum \limits _{i=1}^{n}{A_{i}^{2}}}}{\sqrt {\sum \limits _{i=1}^{n}{B_{i}^{2}}}}}},}
$$
Для облегчения задачи нормализуем вектора рейтингов для каждого пользователя.
$$
R^\prime=r_{ui}-\mu-b_u-b_i.
$$

In [ ]:
train_norm = train_dense_masked - baseline_predictions
train_norm

Вычислим косинусную близость между пользователями и между аниме. Стоит помнить, что функция `cosine_similarity` вычисляет косинусную близость между всеми строками данной матрицы. В `train_norm` в строках лежат данные о пользователях, поэтому `cosine_similarity(train_norm)` даст нам матрицу косинусной близости между всеми пользователями. Для того, чтобы получить в строчках инфорацию об аниме, транспонируем матрицу `train_norm`: `train_norm.T`, и вызовем функцию для неё: `cosine_similarity(train_norm.T)`.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
user_similarities = cosine_similarity(train_norm)
item_similarities = cosine_similarity(train_norm.T)

In [ ]:
assert user_similarities.shape == (len(user_to_idx), len(user_to_idx))
assert item_similarities.shape == (len(anime_to_idx), len(anime_to_idx))

Напишем функцию, которая возвращает `top_k` аниме, похожих на `anime_name`. Заметим, что они будут похожи не по жанру, а по аудитории.

In [ ]:
def top_anime(anime_name, top_k=10):
    # найдем индекс аниме
    anime_idx = anime_to_idx[anime_name]
    
    print('Top {} to: {}'.format(top_k, anime_name))
    # выведем top_k похожих аниме, при этом стоит помнить, что "самое похожее" аниме на anime_name и есть само anime_name
    for i, j in enumerate(np.argsort(item_similarities[anime_idx])[::-1][1:top_k + 1]):
        print('#{}: {}'.format(i + 1, idx_to_anime[j]))

In [ ]:
top_anime('Naruto')

In [ ]:
top_anime('Bleach')

In [ ]:
top_anime('Cowboy Bebop')

In [ ]:
top_anime('Samurai Champloo')

In [ ]:
train_norm

Использую матрицу близости пользователей найдем самых близких $N$ и по ним определим рейтинг.

In [ ]:
def predicted_rating(item_name, user_id, N=50):
    item_idx = anime_to_idx[item_name]
    user_idx = user_to_idx[user_id]
    
    sim_users = np.argsort(user_similarities[user_idx])[::-1][1:N + 1]
    user_values = user_similarities[user_idx][sim_users]
    
    b = train_norm[sim_users].mean(1)
    
    return train_norm[user_idx].mean() + ((train_norm[sim_users, item_idx] - b)*user_values).sum()/np.abs(user_values).sum()

In [ ]:
from tqdm import tqdm

def line_on_user_user(user, top_k=10, N=50):
    ratings = []
    for anime in tqdm(anime_to_idx):
        ratings.append(predicted_rating(anime, user, N))
        
    anime_to_line_on = []
    for anime_idx in np.argsort(ratings)[::-1]:
        if len(anime_to_line_on) == top_k:
            break
        if ma.is_masked(ratings[anime_idx]):
            continue
        if idx_to_anime[anime_idx] not in train_grby_user.get_group(user).name.values:
            anime_to_line_on.append((idx_to_anime[anime_idx], ratings[anime_idx]))
            
    return anime_to_line_on

In [ ]:
user = 18

In [ ]:
train_grby_user.get_group(user)

In [ ]:
dev_grby_user.get_group(user)

In [ ]:
top_k = 10
line_on_user_user(user, top_k=top_k)  # with N=1000 ok!

## Латентные признаки: SVD

Как было сказано на лекции, мы можем использовать для предсказания знания о пользователе, которые он сам сообщил. Это может быть пол, возраст, любимые фильмы и тд. Но что делать, если пользователь ничего не указал? В этом случае нам могут помочь латентные (скрытые) признаки. Рассмотрим пример.
<table>
<tr>
<td></td>
<td>Титаник</td>
<td>Дневник памяти</td>
<td>Трансформеры</td>
<td>Youtube Rewind 2018</td>
<td>Форсаж</td>
</tr>
<tr>
<td>User 1</td>
<td>5</td>
<td>5</td>
<td>1</td>
<td>0</td>
<td>1</td>
</tr>
<tr>
<td>User 2</td>
<td>1</td>
<td>1</td>
<td>5</td>
<td>0</td>
<td>5</td>
</tr>
</table>
Видно, что User 1 предпочитает больше мелодраммы, тогда как User 2 любит больше экшн фильмы, хотя ни тот, ни другой нигде это не указали. Это и есть латентные признаки.

Один из способов "достать" такие признаки - разложение (факторизация) матрицы
$$
R=UI^T,
$$
где $R\in\mathbb{R}^{u\times{}i}$, $U\in\mathbb{R}^{u\times{}k}$ и $I\in\mathbb{R}^{i\times{}k}$, где $k$ небольшое значение. Такого разложения можно добиться, используя `SVD` разложение.

In [ ]:
u, s, vt = scp.linalg.svds(scp.csr_matrix(train_norm.data), k=100)
u = u.dot(np.diag(s))

In [ ]:
def line_on_svd(user, top_k=10):
    ratings = []
    for anime in tqdm(anime_to_idx):
        rat = np.sum(u[user_to_idx[user]]*vt[:, anime_to_idx[anime]])
#         rat *= data_norm_fro
#         rat += bu[user_to_idx[user]]
#         rat += bi[anime_to_idx[anime]]
#         rat += mu
        ratings.append(rat)

    anime_to_line_on = []
    for anime_idx in np.argsort(ratings)[::-1]:
        if ma.is_masked(ratings[anime_idx]):
            continue
        if len(anime_to_line_on) == top_k: break
        if idx_to_anime[anime_idx] not in train_data.groupby('user_id').get_group(user).name.values:
            anime_to_line_on.append((idx_to_anime[anime_idx], ratings[anime_idx]))
            
    return anime_to_line_on

In [ ]:
user = 18
train_grby_user.get_group(user)

In [ ]:
dev_grby_user.get_group(user)

In [ ]:
line_on_svd(user, top_k=top_k)

In [ ]:
line_on_user_user(user, top_k=top_k)

Попробуем использовать библиотеку [`surprise`](https://surprise.readthedocs.io/en/stable/) для SVD разложения.

In [ ]:
from surprise import Reader, Dataset

In [ ]:
# to load dataset from pandas df, we need `load_fromm_df` method in surprise lib
ratings_dict = {'itemID': list(train_data.name),
                'userID': list(train_data.user_id),
                'rating': list(train_data.user_rating)}
df = pd.DataFrame(ratings_dict)
df.head()

In [ ]:
# A reader is still needed but only the rating_scale param is required.
# The Reader class is used to parse a file containing ratings.
reader = Reader(rating_scale=(1, 10))

In [ ]:
# The columns must correspond to user id, item id and ratings (in that order).
data_sur = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)

In [ ]:
from collections import defaultdict

from surprise import SVD

# see https://surprise.readthedocs.io/en/stable/FAQ.html
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [ ]:
# First train an SVD algorithm on the movielens dataset.
# data = Dataset.load_builtin('ml-100k')
trainset = data_sur.build_full_trainset()
algo = SVD()
algo.fit(trainset)

In [ ]:
user = 18
train_data.groupby('user_id').get_group(user)

In [ ]:
dev_data.groupby('user_id').get_group(user)

In [ ]:
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
predictions = algo.test([(user, anime, 0) for anime in anime_to_idx])

In [ ]:
top_k = 10
line_on_user_user(user, top_k)

In [ ]:
line_on_svd(user, top_k)

In [ ]:
top_n = get_top_n(predictions, n=top_k)
top_n[user]